In [ ]:
%reset -f
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

pd.set_option("display.max_rows", 8)
plt.rcParams['figure.figsize'] = (9, 6)

# Create a [DataFrame](https://pandas.pydata.org/pandas-docs/stable/dsintro.html#dataframe)

In [ ]:
dates = pd.date_range('20130101', periods=6)
pd.DataFrame(np.random.randn(6,4), index=dates, columns=list('ABCD'))

In [ ]:
pd.DataFrame({'A' : 1.,
              'B' : pd.Timestamp('20130102'),
              'C' : pd.Series(1,index=list(range(4)),dtype='float32'),
              'D' : np.arange(4,dtype='int32'),
              'E' : pd.Categorical(["test","train","test","train"]),
              'F' : 'foo' })


# Load Data from CSV File

In [ ]:
url = "https://www.fun-mooc.fr/c4x/agrocampusouest/40001S03/asset/AnaDo_JeuDonnees_TemperatFrance.csv"
french_cities = pd.read_csv(url, delimiter=";", encoding="latin1", index_col=0)
french_cities

# Viewing Data

In [ ]:
french_cities.head()

In [ ]:
french_cities.tail()

# Index

In [ ]:
french_cities.index

We can rename an index by setting its name.

In [ ]:
french_cities.index.name = "City"
french_cities.head()

### Exercise 14.1

- Rename DataFrame Months in English
- DataFrame columns are defined in a dictionary.

In [ ]:
import locale
import calendar

locale.setlocale(locale.LC_ALL,'en_US')

months = calendar.month_abbr
print(*months)

# From a local or remote HTML file
We can download and extract data about mean sea level stations around the world from the [PSMSL website](http://www.psmsl.org/).

In [ ]:
# Needs `lxml`, `beautifulSoup4` and `html5lib` python packages
table_list = pd.read_html("http://www.psmsl.org/data/obtaining/")

In [ ]:
# there is 1 table on that page which contains metadata about the stations where 
# sea levels are recorded
local_sea_level_stations = table_list[0]
local_sea_level_stations

# Indexing on DataFrames

In [ ]:
french_cities['Lati']  # DF [] accesses columns (Series)

`.loc` and `.iloc` allow to access individual values, slices or masked selections:

In [ ]:
french_cities.loc['Rennes', "Sep"]

In [ ]:
french_cities.loc['Rennes', ["Sep", "Dec"]]

In [ ]:
french_cities.loc['Rennes', "Sep":"Dec"]

# Masking

In [ ]:
mask = [True, False] * 6
print(french_cities.iloc[0, mask])
print(french_cities.loc["Rennes", mask])

# New column


In [ ]:
french_cities["std"] = french_cities.iloc[:,:12].std(axis=1)
french_cities

In [ ]:
french_cities = french_cities.drop("std", axis=1) # remove this new column

In [ ]:
french_cities

# Modifying a dataframe with multiple indexing

In [ ]:
# french_cities['Rennes']['Sep'] = 25 # It does not works and breaks the DataFrame
french_cities.loc['Rennes']['Sep'] # = 25 is the right way to do it

In [ ]:
french_cities

# Transforming datasets

In [ ]:
french_cities['Mean'].min(), french_cities['Ampl'].max()

## Apply

Let's convert the temperature mean from Celsius to Fahrenheit degree.

In [ ]:
fahrenheit = lambda T: T*9/5+32
french_cities['Mean'].apply(fahrenheit)

## Sort

In [ ]:
french_cities.sort_values(by='Lati')

In [ ]:
french_cities = french_cities.sort_values(by='Lati',ascending=False)
french_cities

## Stack and unstack

Instead of seeing the months along the axis 1, and the cities along the axis 0, let's try to convert these into an outer and an inner axis along only 1 time dimension.

In [ ]:
pd.set_option("display.max_rows", 20)
unstacked = french_cities.iloc[:,:12].unstack()
unstacked

In [ ]:
type(unstacked)

## Transpose

The result is grouped in the wrong order since it sorts first the axis that was unstacked. We need to transpose the dataframe.

In [ ]:
city_temp = french_cities.iloc[:,:12].transpose()
city_temp.plot()

In [ ]:
city_temp.boxplot(rot=90);

# Describing

In [ ]:
french_cities['Région'].describe()

In [ ]:
french_cities['Région'].unique()

In [ ]:
french_cities['Région'].value_counts()

In [ ]:
# To save memory, we can convert it to a categorical column:
french_cities["Région"] = french_cities["Région"].astype("category")

In [ ]:
french_cities.memory_usage()

# Data Aggregation/summarization

## groupby

In [ ]:
fc_grouped_region = french_cities.groupby("Région")
type(fc_grouped_region)

In [ ]:
for group_name, subdf in fc_grouped_region:
    print(group_name)
    print(subdf)
    print("")

## Transferring R data sets into Python

In [ ]:
%load_ext rpy2.ipython

In [ ]:
%%R
library('missMDA')
data(orange)
estim_ncpPCA(orange)

- conversions of R to pandas objects will be done automatically

In [ ]:
from rpy2.robjects import r, pandas2ri
pandas2ri.activate()
r.library('missMDA')
r.data('orange')
orange = r('orange')

In [ ]:
orange

In [ ]:
from rpy2.robjects.packages import importr
missMDA = importr("missMDA")

In [ ]:
missMDA.estim_ncpPCA(orange)

In [ ]:
res = missMDA.imputePCA(orange,ncp=2)

In [ ]:
res[0]

In [ ]:
import rpy2
pd_dataframe = pd.DataFrame(np.array(res[0]),columns=orange.columns)

In [ ]:
pd_dataframe

In [ ]:
r('library(missMDA)')
r('df <- imputePCA(orange,ncp=2) ')
r('res <- as.data.frame(df$completeObs)')
orange = r('res')
orange